In [ ]:
import numpy as np
import laspy
import rasterio
from rasterio.transform import from_origin
from rasterio.fill import fillnodata

# --- inputs / outputs ---
infile   = r"C:\Users\usuario\Downloads\test_las_orlando.las"
out_raw  = r"C:\Users\usuario\Downloads\test_las_orlando.tif"
out_fill = r"C:\Users\usuario\Downloads\test_las_orlando_filled.tif"

# EPSG:6438 is in feet; set pixel size in feet
res    = 1.0          # try 1.0–2.0 ft if you still see tiny holes
nodata = -9999.0

# --- read points ---
las = laspy.read(infile)
x = np.asarray(las.x); y = np.asarray(las.y); z = np.asarray(las.z, dtype=np.float32)

# Use ALL returns; just drop 'noise' (class 7) if present
mask = np.ones(x.size, dtype=bool)
if "classification" in las.point_format.dimension_names:
    mask &= (np.asarray(las.classification) != 7)

x = x[mask]; y = y[mask]; z = z[mask]

# --- grid geometry (snap origin to resolution) ---
xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()

origin_x = np.floor(xmin / res) * res
origin_y = np.ceil(ymax / res) * res

cols = int(np.ceil((xmax - origin_x) / res))
rows = int(np.ceil((origin_y - ymin) / res))

# pixel indices
ci = np.floor((x - origin_x) / res).astype(np.int32)
ri = np.floor((origin_y - y) / res).astype(np.int32)

# keep only in-bounds points
valid = (ci >= 0) & (ci < cols) & (ri >= 0) & (ri < rows)
ci = ci[valid]; ri = ri[valid]; z = z[valid]

# --- accumulate DSM = cell-wise maximum ---
accum = np.full((rows, cols), -np.inf, dtype=np.float32)
np.maximum.at(accum, (ri, ci), z)

grid = accum
grid[~np.isfinite(grid)] = nodata

transform = from_origin(origin_x, origin_y, res, res)

# --- write RAW DSM (with nodata gaps) ---
with rasterio.open(
    out_raw, "w",
    driver="GTiff", height=rows, width=cols, count=1,
    dtype="float32", nodata=nodata, transform=transform, crs="EPSG:6438",
    compress="deflate", tiled=True
) as dst:
    dst.write(grid, 1)

# --- fill remaining NoData by interpolation ---
mask_nodata = (grid == nodata)
grid_filled = fillnodata(
    grid.copy(),
    mask=mask_nodata,           # pixels to fill (True = NoData)
    max_search_distance=5,      # increase to bridge wider gaps (in pixels)
    smoothing_iterations=1      # light smoothing of inpainted areas
)

with rasterio.open(
    out_fill, "w",
    driver="GTiff", height=rows, width=cols, count=1,
    dtype="float32", nodata=nodata, transform=transform, crs="EPSG:6438",
    compress="deflate", tiled=True
) as dst:
    dst.write(grid_filled, 1)

print(f"Written:\n  RAW   -> {out_raw}\n  FILLED-> {out_fill}")
